# Advanced Financial Model - PyProforma v2 API

This notebook demonstrates an advanced PyProforma v2 financial model with:
- Fixed line items
- Formula lines with dependencies
- Assumptions
- Time-offset lookback references (e.g., `revenue[-1]`)
- Value overrides

The model calculates:
- Revenue with year-over-year growth
- Operating expenses as a percentage of revenue
- EBITDA (revenue - expenses)
- Year-over-year growth rate
- Tax expense and net income

## Import Required Libraries

Import the necessary PyProforma v2 components.

In [1]:
from pyproforma.v2 import Assumption, FixedLine, FormulaLine, ProformaModel

## Define the AdvancedFinancialModel Class

Create a class that inherits from `ProformaModel` to define the advanced financial model structure. This includes:
- **Assumptions**: Operating expense ratio and corporate tax rate
- **Revenue**: Fixed values for years 2024-2027
- **Operating Expenses**: Calculated as revenue × expense ratio
- **EBITDA**: Revenue - Operating Expenses
- **Tax Expense**: EBITDA × tax rate
- **Net Income**: EBITDA - Tax Expense
- **YoY Growth**: Year-over-year growth with time-offset lookback

In [2]:
class AdvancedFinancialModel(ProformaModel):
    """
    A more advanced financial model demonstrating time offsets and lookbacks.

    This model calculates:
    - Revenue with year-over-year growth
    - Operating expenses as a percentage of revenue
    - EBITDA (revenue - expenses)
    - Year-over-year growth rate
    - Cumulative profit
    """

    # Define assumptions
    expense_ratio = Assumption(value=0.55, label="Operating Expense Ratio")
    tax_rate = Assumption(value=0.21, label="Corporate Tax Rate")

    # Calculate tax expense
    tax_expense = FormulaLine(
        formula=lambda a, li, t: li.ebitda[t] * a.tax_rate,
        label="Tax Expense",
    )

    # Define base revenue for first year
    revenue = FixedLine(
        values={2024: 100000, 2025: 115000, 2026: 132000, 2027: 152000},
        label="Revenue",
    )

    # Calculate operating expenses
    operating_expenses = FormulaLine(
        formula=lambda a, li, t: li.revenue[t] * a.expense_ratio,
        label="Operating Expenses",
    )

    # Calculate EBITDA
    ebitda = FormulaLine(
        formula=lambda a, li, t: li.revenue[t] - li.operating_expenses[t],
        label="EBITDA",
    )

    # Calculate net income
    net_income = FormulaLine(
        formula=lambda a, li, t: li.ebitda[t] - li.tax_expense[t],
        label="Net Income",
    )

    # Calculate YoY growth rate (with override for first year)
    yoy_growth = FormulaLine(
        formula=lambda a, li, t: (li.revenue[t] - li.revenue[t - 1])
        / li.revenue[t - 1],
        values={2024: 0.0},  # First year has no prior year
        label="YoY Revenue Growth %",
    )

## Instantiate the Model

Create an instance of the `AdvancedFinancialModel` with periods for years 2024-2027.

In [3]:
# Create the model
model = AdvancedFinancialModel(periods=[2024, 2025, 2026, 2027])

## Access Line Item Results with Dictionary-Style Syntax

The v2 API supports dictionary-style access to line items using `model['line_item_name']`. This returns a `LineItemResult` object that provides convenient access to:
- **Values across all periods**: Access using indexing (`result[year]`) or iteration
- **Metadata**: Label, formula, and other line item properties
- **Dictionary conversion**: Convert to a dict with `result.to_dict()`

In [4]:
# Access line items using dictionary-style syntax
net_income_result = model['net_income']
revenue_result = model['revenue']
ebitda_result = model['ebitda']

print("=" * 60)
print("LineItemResult Examples - Dictionary-Style Access")
print("=" * 60)
print()

# Display metadata
print(f"Line Item Name: {net_income_result.name}")
print(f"Line Item Label: {net_income_result.label}")
print(f"Type: {type(net_income_result).__name__}")
print()

# Access individual year values using indexing
print("Net Income by Year (using indexing):")
for year in model.periods:
    print(f"  {year}: ${net_income_result[year]:>12,.0f}")
print()

# Get all values as a dictionary
print("Net Income as Dictionary (using .values property):")
print(f"  {net_income_result.values}")
print()

LineItemResult Examples - Dictionary-Style Access

Line Item Name: net_income
Line Item Label: Net Income
Type: LineItemResult

Net Income by Year (using indexing):
  2024: $      35,550
  2025: $      40,882
  2026: $      46,926
  2027: $      54,036

Net Income as Dictionary (using .values property):
  {2024: 35549.99999999999, 2025: 40882.49999999999, 2026: 46926.0, 2027: 54036.0}



## Working with LineItemResult Objects

LineItemResult objects support iteration and can be used in calculations and comparisons.

In [5]:
# Iterate over values using .values property
print("Iterating over Revenue values:")
for year, value in revenue_result.values.items():
    print(f"  {year}: ${value:>12,.0f}")
print()

# Calculate totals and averages
total_revenue = sum(revenue_result.values.values())
avg_revenue = total_revenue / len(model.periods)

print(f"Total Revenue (all years): ${total_revenue:>12,.0f}")
print(f"Average Revenue:           ${avg_revenue:>12,.0f}")
print()

# Calculate growth using LineItemResult objects
print("Revenue Growth Analysis:")
revenue_values_list = list(revenue_result.values.values())
for i, year in enumerate(model.periods[1:], start=1):
    growth = (revenue_values_list[i] - revenue_values_list[i-1]) / revenue_values_list[i-1]
    print(f"  {year}: {growth:>7.1%}")
print()

Iterating over Revenue values:
  2024: $     100,000
  2025: $     115,000
  2026: $     132,000
  2027: $     152,000

Total Revenue (all years): $     499,000
Average Revenue:           $     124,750

Revenue Growth Analysis:
  2025:   15.0%
  2026:   14.8%
  2027:   15.2%



## Access Assumptions with Dictionary-Style Syntax

You can also access assumptions using `model['assumption_name']`, which returns an `AssumptionResult` object with similar properties to `LineItemResult`.

In [6]:
# Access assumptions using dictionary-style syntax
expense_ratio_result = model['expense_ratio']
tax_rate_result = model['tax_rate']

print("=" * 60)
print("AssumptionResult Examples - Dictionary-Style Access")
print("=" * 60)
print()

# Display metadata
print(f"Assumption Name: {expense_ratio_result.name}")
print(f"Assumption Label: {expense_ratio_result.label}")
print(f"Assumption Value: {expense_ratio_result.value}")
print(f"Type: {type(expense_ratio_result).__name__}")
print()

# Access both assumptions
print("Model Assumptions:")
print(f"  {tax_rate_result.label}: {tax_rate_result.value:.1%}")
print(f"  {expense_ratio_result.label}: {expense_ratio_result.value:.1%}")
print()

# Convert to float for calculations
print("Using AssumptionResult in calculations:")
print(f"  If revenue is $100,000:")
print(f"  Expenses = $100,000 × {float(expense_ratio_result):.0%} = ${100000 * float(expense_ratio_result):,.0f}")
print()

AssumptionResult Examples - Dictionary-Style Access

Assumption Name: expense_ratio
Assumption Label: Operating Expense Ratio
Assumption Value: 0.55
Type: AssumptionResult

Model Assumptions:
  Corporate Tax Rate: 21.0%
  Operating Expense Ratio: 55.0%

Using AssumptionResult in calculations:
  If revenue is $100,000:
  Expenses = $100,000 × 55% = $55,000



## Display Model Assumptions

View the configured assumptions using the `model.av` (assumption values) accessor.

In [7]:
print("=" * 60)
print("Advanced Financial Model - v2 API Demonstration")
print("=" * 60)
print()
print("Model Assumptions:")
print(f"  Operating Expense Ratio: {model.av.expense_ratio:.1%}")
print(f"  Corporate Tax Rate: {model.av.tax_rate:.1%}")
print()

Advanced Financial Model - v2 API Demonstration

Model Assumptions:
  Operating Expense Ratio: 55.0%
  Corporate Tax Rate: 21.0%



## Display Income Statements by Year

Loop through each period and display a formatted income statement. This demonstrates both access methods:
- **Dictionary-style**: `model['line_item_name'][year]`
- **Traditional method**: `model.li.get('line_item_name', year)`

In [8]:
# Display income statement for each year using dictionary-style access
for year in model.periods:
    print(f"{year} Income Statement (using model['item'][year]):")
    print(f"  Revenue:              ${model['revenue'][year]:>12,.0f}")
    print(f"  Operating Expenses:   ${model['operating_expenses'][year]:>12,.0f}")
    print(f"  EBITDA:               ${model['ebitda'][year]:>12,.0f}")
    print(f"  Tax Expense:          ${model['tax_expense'][year]:>12,.0f}")
    print(f"  Net Income:           ${model['net_income'][year]:>12,.0f}")
    print()

# Alternative: using traditional model.li.get() method
print("Alternative syntax - using model.li.get():")
year = model.periods[0]  # Show first year as example
print(f"{year} Income Statement (using model.li.get('item', year)):")
print(f"  Revenue:              ${model.li.get('revenue', year):>12,.0f}")
print(f"  Net Income:           ${model.li.get('net_income', year):>12,.0f}")
print()

2024 Income Statement (using model['item'][year]):
  Revenue:              $     100,000
  Operating Expenses:   $      55,000
  EBITDA:               $      45,000
  Tax Expense:          $       9,450
  Net Income:           $      35,550

2025 Income Statement (using model['item'][year]):
  Revenue:              $     115,000
  Operating Expenses:   $      63,250
  EBITDA:               $      51,750
  Tax Expense:          $      10,867
  Net Income:           $      40,882

2026 Income Statement (using model['item'][year]):
  Revenue:              $     132,000
  Operating Expenses:   $      72,600
  EBITDA:               $      59,400
  Tax Expense:          $      12,474
  Net Income:           $      46,926

2027 Income Statement (using model['item'][year]):
  Revenue:              $     152,000
  Operating Expenses:   $      83,600
  EBITDA:               $      68,400
  Tax Expense:          $      14,364
  Net Income:           $      54,036

Alternative syntax - using model

## Display Growth Metrics

Display the year-over-year growth percentage for revenue, demonstrating the time-offset lookback feature.

In [9]:
# Display growth metrics
print("Growth Metrics:")
for year in model.periods:
    growth = model.li.get("yoy_growth", year)
    print(f"  {year} YoY Growth:        {growth:>7.1%}")
print()

print("=" * 60)
print("Model successfully calculated using v2 API!")
print("=" * 60)

Growth Metrics:
  2024 YoY Growth:           0.0%
  2025 YoY Growth:          15.0%
  2026 YoY Growth:          14.8%
  2027 YoY Growth:          15.2%

Model successfully calculated using v2 API!


## Complete Model as Table


In [10]:
model.tables.line_items(include_label=True).show()


Name,Label,2024,2025,2026,2027
tax_expense,Tax Expense,"9,450","10,867","12,474","14,364"
revenue,Revenue,"100,000","115,000","132,000","152,000"
operating_expenses,Operating Expenses,"55,000","63,250","72,600","83,600"
ebitda,EBITDA,"45,000","51,750","59,400","68,400"
net_income,Net Income,"35,550","40,882","46,926","54,036"
yoy_growth,YoY Revenue Growth %,0,0,0,0
,Total,"245,000","281,750","323,400","372,400"
